In [1]:
import numpy as np
from numpy.random import randn
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_theme()
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 50)

### Use data from year 2017 - 2019 - named df1
From the EDA, resale prices remain relatively stable during this time period. To isolate the effect of other features on HDB price, we will study only data between this period.

In [2]:
df=pd.read_csv('HDB_prices_v2.csv')
df.drop('Unnamed: 0', axis=1, inplace =True)

In [3]:
df1 = df[df['year'].isin([2017,2018,2019])]

### Feature Engineering: Distance to Common Amenities
We create new features: distance from flat to common amenities.

Amenities are:
  1. Schools
  2. Hawker Centers
  3. Supermarkets
  4. MRT stations
  5. Central Area(Raffles Place MRT)

The respective datasets were downloaded from data.gov.sg. Onemap.sg api was used to obtain their coordinates. These coordinates were fed through other functions which utilize geopy package to obtain distance between two locations. The script is called 'Get_coordinates'.

The results were additional information on distance to the various amenities for each unique house address. They were are located in the file 'flat_amenities.csv'.

### Merge amenities data to flat data

In [4]:
## Read in flat amenities
flat_amenities = pd.read_csv('data/flat_amenities.csv')
#Combine street_name and block to make 'flat' column
df1['flat'] = df1['block'] + ' ' + df1['street_name']
# Merge flat_amenities based 
df1 = df1.merge(flat_amenities, on = 'flat', how ='left' )
##Drop unrequired columns
df1.drop(['school','hawker', 'mrt', 'date', 'year', 'block', 'street_name', 'cpi', 'lease_commence_date',
        'resale_price'], axis = 1, inplace = True)
##Rearrange columns 
cols_at_end = ['Price_per_SF', 'real_price']
df1 = df1[[c for c in df1 if c not in cols_at_end] 
        + [c for c in cols_at_end if c in df1]]

In [5]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 64256 entries, 0 to 64255
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   town                 64256 non-null  object 
 1   flat_type            64256 non-null  object 
 2   storey_range         64256 non-null  object 
 3   floor_area_sqm       64256 non-null  float64
 4   flat_model           64256 non-null  object 
 5   remaining_lease      64256 non-null  int64  
 6   flat                 64256 non-null  object 
 7   school_dist          64103 non-null  float64
 8   num_school_2km       64103 non-null  float64
 9   hawker_dist          64103 non-null  float64
 10  num_hawker_2km       64103 non-null  float64
 11  mrt_dist             64103 non-null  float64
 12  num_mrt_2km          64103 non-null  float64
 13  supermarket_dist     64103 non-null  float64
 14  num_supermarket_2km  64103 non-null  float64
 15  dist_central         64103 non-null 

In [7]:
df1.head(2)

,town,flat_type,storey_range,floor_area_sqm,flat_model,remaining_lease,flat,school_dist,num_school_2km,hawker_dist,num_hawker_2km,mrt_dist,num_mrt_2km,supermarket_dist,num_supermarket_2km,dist_central,Price_per_SF,real_price
0,Queenstown,4 ROOM,04 TO 06,88.0,Improved,56,2 HOLLAND AVE,1.203077,6.0,0.214065,5.0,0.242371,8.0,0.300772,15.0,7.109491,6773.368339,596056.413855
1,Queenstown,4 ROOM,28 TO 30,92.0,Model A,95,23 GHIM MOH LINK,0.849438,4.0,0.329263,6.0,0.459740,7.0,0.054650,15.0,7.736062,7367.106798,677773.825432


### Clean up missing values in df1

In [6]:
df1.isnull().sum()

town                     0
flat_type                0
storey_range             0
floor_area_sqm           0
flat_model               0
remaining_lease          0
flat                     0
school_dist            153
num_school_2km         153
hawker_dist            153
num_hawker_2km         153
mrt_dist               153
num_mrt_2km            153
supermarket_dist       153
num_supermarket_2km    153
dist_central           153
Price_per_SF             0
real_price               0
dtype: int64

In [7]:
df1[df1['school_dist'].isna()]['town'].value_counts()

Kallang/Whampoa    153
Name: town, dtype: int64

In [5]:
temp = df1.copy()
col = list(df1.columns[-11:])
## fill in median value for each distance feature, based on town = Kallang/Whampoa
for feat in col:
    temp[feat] = temp[feat].fillna(temp.groupby('town')[feat].transform('median'))
temp.isnull().sum()

town                   0
flat_type              0
storey_range           0
floor_area_sqm         0
flat_model             0
remaining_lease        0
flat                   0
school_dist            0
num_school_2km         0
hawker_dist            0
num_hawker_2km         0
mrt_dist               0
num_mrt_2km            0
supermarket_dist       0
num_supermarket_2km    0
dist_central           0
Price_per_SF           0
real_price             0
dtype: int64

In [6]:
df1 = temp.copy()

## Additional cleaning: Flat_model and Town
From the EDA earlier, flat_model, towns have too many categories. Let's combine them to larger groups based on similar items.

We will also remove those classes from flat_model and flat_type that have very few instances.

In [7]:
## Flat Types: remove 1 room and MultiGen
df1.drop(df1[(df1.flat_type == '1 ROOM') | (df1.flat_type == 'MULTI GENERATION')].index, inplace = True)

In [11]:
df1['flat_model'].value_counts()

Model A                   20743
Improved                  16058
New Generation             9061
Premium Apartment          6945
Simplified                 2754
Apartment                  2627
Maisonette                 1909
Standard                   1802
Dbss                        936
Model A2                    885
Adjoined Flat               119
Type S1                     117
Model A-Maisonette          110
Type S2                      63
Terrace                      38
Improved-Maisonette          13
Premium Apartment Loft        7
Premium Maisonette            7
Name: flat_model, dtype: int64

In [8]:
## Flat Model: group them into larger classes
replace_values = {'Model A-Maisonette':'Maisonette','Improved-Maisonette':'Maisonette', 'Premium Maisonette':'Maisonette',
                  'Terrace':'Special', 'Adjoined Flat':'Special', 'Type S1':'Special', 'Type S2':'Special', 'Dbss':'Special', 
                  'Model A2':'Model A', 
                  'Premium Apartment Loft':'Apartment', 
                  'Improved':'Standard', 'Simplified':'Model A', '2-Room':'Standard'}
df1 = df1.replace({'flat_model': replace_values})

In [9]:
df1['flat_model'].value_counts()

Model A              24382
Standard             17860
New Generation        9061
Premium Apartment     6945
Apartment             2634
Maisonette            2039
Special               1273
Name: flat_model, dtype: int64

In [10]:
# Group towns into regions
df1['town'] =df1['town'].apply(lambda x: x.upper())
d_region = {'ANG MO KIO':'North East', 'BEDOK':'East', 'BISHAN':'Central', 'BUKIT BATOK':'West', 'BUKIT MERAH':'Central',
       'BUKIT PANJANG':'West', 'BUKIT TIMAH':'Central', 'CENTRAL AREA':'Central', 'CHOA CHU KANG':'West',
       'CLEMENTI':'West', 'GEYLANG':'Central', 'HOUGANG':'North East', 'JURONG EAST':'West', 'JURONG WEST':'West',
       'KALLANG/WHAMPOA':'Central', 'MARINE PARADE':'Central', 'PASIR RIS':'East', 'PUNGGOL':'North East',
       'QUEENSTOWN':'Central', 'SEMBAWANG':'North', 'SENGKANG':'North East', 'SERANGOON':'North East', 'TAMPINES':'East',
       'TOA PAYOH':'Central', 'WOODLANDS':'North', 'YISHUN':'North'}
df1['region'] = df1['town'].map(d_region)

In [11]:
##Rearrange columns 
cols_at_end = ['Price_per_SF', 'real_price']
df1 = df1[[c for c in df1 if c not in cols_at_end] 
        + [c for c in cols_at_end if c in df1]]
df1.head()

,town,flat_type,storey_range,floor_area_sqm,flat_model,remaining_lease,flat,school_dist,num_school_2km,hawker_dist,num_hawker_2km,mrt_dist,num_mrt_2km,supermarket_dist,num_supermarket_2km,dist_central,region,Price_per_SF,real_price
0,QUEENSTOWN,4 ROOM,04 TO 06,88.0,Standard,56,2 HOLLAND AVE,1.203077,6.0,0.214065,5.0,0.242371,8.0,0.300772,15.0,7.109491,Central,6773.368339,596056.413855
1,QUEENSTOWN,4 ROOM,28 TO 30,92.0,Model A,95,23 GHIM MOH LINK,0.849438,4.0,0.329263,6.0,0.459740,7.0,0.054650,15.0,7.736062,Central,7367.106798,677773.825432
2,QUEENSTOWN,4 ROOM,04 TO 06,82.0,Standard,56,8 HOLLAND AVE,0.977393,5.0,0.519236,5.0,0.395342,9.0,0.348531,14.0,6.831752,Central,5920.701987,485497.562898
3,QUEENSTOWN,4 ROOM,19 TO 21,88.0,Standard,57,11 HOLLAND DR,0.198307,5.0,0.966308,3.0,0.816467,5.0,0.355063,16.0,8.503199,Central,6609.496525,581635.694165
4,QUEENSTOWN,4 ROOM,10 TO 12,103.0,Model A,78,130 CLARENCE LANE,0.749409,7.0,0.981904,7.0,0.413269,3.0,0.430021,20.0,4.182996,Central,7280.363339,749877.423883


In [12]:
df1.to_csv('HDB 1719 with Distance Features.csv', index = False)

### Perform some Analysis on the new Features
1. New Distance Features
2. Region Feature
3. Flat Model

In [17]:
df1.head()


,town,flat_type,storey_range,floor_area_sqm,flat_model,remaining_lease,flat,school_dist,num_school_2km,hawker_dist,num_hawker_2km,mrt_dist,num_mrt_2km,supermarket_dist,num_supermarket_2km,dist_central,region,Price_per_SF,real_price
0,QUEENSTOWN,4 ROOM,04 TO 06,88.0,Standard,56,2 HOLLAND AVE,1.203077,6.0,0.214065,5.0,0.242371,8.0,0.300772,15.0,7.109491,Central,6773.368339,596056.413855
1,QUEENSTOWN,4 ROOM,28 TO 30,92.0,Model A,95,23 GHIM MOH LINK,0.849438,4.0,0.329263,6.0,0.459740,7.0,0.054650,15.0,7.736062,Central,7367.106798,677773.825432
2,QUEENSTOWN,4 ROOM,04 TO 06,82.0,Standard,56,8 HOLLAND AVE,0.977393,5.0,0.519236,5.0,0.395342,9.0,0.348531,14.0,6.831752,Central,5920.701987,485497.562898
3,QUEENSTOWN,4 ROOM,19 TO 21,88.0,Standard,57,11 HOLLAND DR,0.198307,5.0,0.966308,3.0,0.816467,5.0,0.355063,16.0,8.503199,Central,6609.496525,581635.694165
4,QUEENSTOWN,4 ROOM,10 TO 12,103.0,Model A,78,130 CLARENCE LANE,0.749409,7.0,0.981904,7.0,0.413269,3.0,0.430021,20.0,4.182996,Central,7280.363339,749877.423883


In [3]:
df = pd.read_csv('HDB 1719 with Distance Features.csv')

In [7]:
df['flat_type'].value_counts()

4 ROOM       26601
5 ROOM       15916
3 ROOM       15589
EXECUTIVE     5169
2 ROOM         919
Name: flat_type, dtype: int64